# Site Monitoring and Contour Generation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import geopandas as gpd
from tensorflow import keras
from tqdm.notebook import tqdm

from scripts import contour_gen

## Load models and set parameters

In [ ]:
model_name = 'spectrogram_v0.0.11_2021-07-13'
model = keras.models.load_model(f'../models/{model_name}.h5')

ensemble_name = 'v0.0.11_ensemble-8-25-21'
model_list = contour_gen.load_ensemble(f'../models/{ensemble_name}')

In [ ]:
version_num = str(2.1)
contour_output_path = f'../data/model_outputs/site_contours/{version_num}'
if not os.path.exists(contour_output_path):
    os.mkdir(contour_output_path)

# Rect needs to be large enough to cover big sites. 
# But large rects take longer to process and increase false positive likelihood
rect_width = 0.01
start_date = '2015-11-01'
end_date = '2022-03-30'
mosaic_period = 2
spectrogram_interval = 3
scale=4

In [ ]:
confirmed_sites_file = 'missing_contours'
confirmed_sites = gpd.read_file(f"../data/site_metadata/{confirmed_sites_file}.geojson")
coords = [[site.x, site.y] for site in confirmed_sites['geometry']]
names = confirmed_sites['id']
print(len(confirmed_sites), 'sites loaded')

In [ ]:
contour_gdf = gpd.GeoDataFrame(columns=['geometry', 'area (km^2)', 'date', 'name']).set_crs('epsg:4326')
failures = []
for coord, name in zip(tqdm(coords), names):
    site = contour_gen.SiteContours(coord, name)
    site.download_pairs(
        start_date=start_date,
        end_date=end_date,
        mosaic_period=mosaic_period,
        spectrogram_interval=spectrogram_interval,
        rect_width=rect_width
    )
    if len(site.pairs) > 0:
        site.generate_predictions(model)
        site.threshold_predictions(threshold=0.5)
        site.mask_predictions(window_size=2, threshold=0.1)
        site.generate_contours(threshold=0.5, scale=scale, plot=False)
        site.generate_polygons(plot=False)
        site.compile_contours()
        contour_gdf = contour_gdf.append(site.contour_gdf, ignore_index=True)



In [ ]:
contour_gdf.to_file(f'{contour_output_path}/{confirmed_sites_file}_contours.geojson', driver='GeoJSON')

In [ ]:
import requests
import datetime
endpoint = 'http://api.plastic.watch.earthrise.media'
site_endpoint = f'{endpoint}/sites/{site.name}/contours'
auth = requests.auth.HTTPBasicAuth('admin', 'plastics')
delete = requests.delete(site_endpoint, auth=auth)
print("delete status", delete.status_code)
site.contour_gdf.to_crs("epsg:3395")
site.contour_gdf['date'] = [datetime.datetime.isoformat(date) for date in site.contour_gdf['date']]
r = requests.post(site_endpoint, site.contour_gdf.to_json())
print("request status", r.status_code)